# PyTorch Introduction

## First steps

(Reference: https://github.com/yunjey/pytorch-tutorial)

Importing PyTorch:

In [3]:
import torch 
import torchvision
import numpy as np
import torchvision.transforms as transforms
from sklearn.metrics import f1_score

Creating tensors:

In [4]:
x = torch.tensor(4., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

Build a computational graph:

In [5]:
y = w * x + b

With the ```backward``` method you can compute the gradients of a tensor:

In [6]:
# Compute gradients.
y.backward()

# Print out the gradients.
print(x.grad)
print(w.grad)
print(b.grad)

tensor(2.)
tensor(4.)
tensor(1.)


You can use the ```randn``` function to create a tensor with random numbers:

In [7]:
# Create tensors of shape (10, 3) and (10, 2).
x = torch.randn(10, 3)
y = torch.randn(10, 2)

The PyTorch library has some pre-defined neural network layer.
They are placed in ```torch.nn.*```.
One example is a fully connected layer.
You can create it as follows:

In [8]:
# Build a fully connected layer.
linear = torch.nn.Linear(3, 2)
print ('w: ', linear.weight)
print ('b: ', linear.bias)

w:  Parameter containing:
tensor([[-0.0538, -0.1775, -0.2113],
        [-0.4158, -0.4134,  0.2922]], requires_grad=True)
b:  Parameter containing:
tensor([ 0.1939, -0.2951], requires_grad=True)


If you want to train your model, you can use one of the losses (e.g. MSELoss) and an optimizer (e.g. SGD, Adam, ...).
You will find an example below:

In [9]:
# Build loss function and optimizer.
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

# Forward pass.
pred = linear(x)

# Compute loss.
loss = criterion(pred, y)
print('loss: ', loss.item())

# 1-step gradient descent.
optimizer.step()

# Print out the loss after 1-step gradient descent.
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.item())

loss:  1.5882165431976318
loss after 1 step optimization:  1.5882165431976318


### Loading data from numpy

Below you will find an example of how to convert a numpy array to a PyTorch tensor and vice versa.

In [10]:
# Create a numpy array.
x = np.array([[1, 2], [3, 4]])

# Convert the numpy array to a torch tensor.
y = torch.from_numpy(x)

# Convert the torch tensor to a numpy array.
z = y.numpy()

### PyTorch Modules

In PyTorch you can define Modules which are basically container of operations.
For Instance ```Linear``` is a module.
However, modules can contain other modules, allowing to nest them in a tree structure.
For instance you create a sequential module (```Sequential```) containing other modules.
```Sequential``` will execute the modules (as the name proposes) sequentially, such that the output of the $n^{th}$ module is the input of the $(n+1)^{th}$ module:

In [11]:
model = torch.nn.Sequential(
    torch.nn.Linear(100, 50),
    torch.nn.Linear(50, 50),
    torch.nn.Linear(50, 100),
)

Alternatively you can define the same module as follows (allowing non-sequential and re-usable modules):

In [12]:
class MyModule(torch.nn.Module):
    def __init__(self):
        super(MyModule, self).__init__()
        self.fc1 = torch.nn.Linear(100, 50)
        self.fc2 = torch.nn.Linear(50, 50)
        self.fc3 = torch.nn.Linear(50, 100)

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

model = MyModule()

If you want to execute on GPU:

In [14]:
model.cuda()(torch.randn(1, 100).cuda())

AssertionError: Torch not compiled with CUDA enabled

### Save and load the model

In [15]:
# Save and load the entire model.
torch.save(model, 'model.ckpt')
model = torch.load('model.ckpt')

# Save and load only the model parameters (recommended).
torch.save(model.state_dict(), 'params.ckpt')
model.load_state_dict(torch.load('params.ckpt'))

/Users/changkun/dev/ws-18-19-deep-learning-tutorial/.env/lib/python3.6/site-packages/torch/serialization.py:250: UserWarning: Couldn't retrieve source code for container of type MyModule. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


## CIFAR10 in PyTorch

As in tensorflow there is a easy way to use some common datasets:

In [16]:
# Download and construct CIFAR-10 dataset.
train_dataset = torchvision.datasets.CIFAR10(root='/tmp/data/',
                                             train=True, 
                                             transform=transforms.ToTensor(),
                                             download=True)
test_dataset = torchvision.datasets.CIFAR10(root='/tmp/data/',
                                            train=False,
                                            transform=transforms.ToTensor(),
                                            download=True)

Files already downloaded and verified


You can access the dataset as usual:

In [17]:
# Fetch one data pair (read data from disk).
image, label = train_dataset[0]
print (image.size())
print (label)

torch.Size([3, 32, 32])
6


PyTorch has a DataLoader class implemented.
This class is handling for you shuffling batch sizes etc:

In [18]:
# Data loader (this provides queues and threads in a very simple way).
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=64, shuffle=False)

When you iterate over the DataLoader, you get (shuffled) batches for training/testing:

In [19]:
# When iteration starts, queue and thread start to load data from files.
data_iter = iter(train_loader)

# Mini-batch images and labels.
images, labels = data_iter.next()

# Actual usage of the data loader is as below.
for images, labels in train_loader:
    # Training code should be written here.
    pass

In the following cell defines a function that trains a model given an optimizer, criterion and a DataLoader:

In [20]:
def train(net, optimizer, criterion, train_loader, epochs=15):
    for epoch in range(epochs):
        def do_epoch():
            loss = 0
            for inputs, labels in train_loader:
                optimizer.zero_grad()
                loss = criterion(net(inputs), labels)
                loss.backward()
                optimizer.step()

                loss += loss.item()
            print("epoch %d -> loss: %.3f" % (epoch, loss))
        #%time do_epoch()
        do_epoch()

In the next cell we define a function that computes the f1 score of a trained model:

In [21]:
def test(net, test_loader):
    y_pred, y_true = [], []
    for inputs, labels in test_loader:
        y_true = np.concatenate([y_true, labels.numpy()])
        _, pred = torch.max(net(inputs), 1)
        y_pred = np.concatenate([y_pred, pred])
    f1 = f1_score(y_true, y_pred, average='macro')
    print(f1)

Below we define the same baseline network as in the CIFAR10 competition:

In [22]:
class Baseline(torch.nn.Module):
    def __init__(self):
        super(Baseline, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 16, 5, padding=2)
        self.conv2 = torch.nn.Conv2d(16, 32, 5, padding=2)
        self.conv3 = torch.nn.Conv2d(32, 64, 5, padding=2)
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = torch.nn.Linear(64 * 4 * 4, 512)
        self.out = torch.nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(torch.nn.functional.relu(self.conv1(x)))
        x = self.pool(torch.nn.functional.relu(self.conv2(x)))
        x = self.pool(torch.nn.functional.relu(self.conv3(x)))
        x = x.view(-1, 64 * 4 * 4)
        x = torch.nn.functional.relu(self.fc1(x))
        x = self.out(x) # as before, softmax is applied together with the CrossEntropyLoss!
        return x


net = Baseline()

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

Train and store the weights:

In [23]:
#net.load_state_dict(torch.load('baseline.ckpt'))
%time train(net, optimizer, criterion, train_loader)
torch.save(net.state_dict(), 'baseline.ckpt')

epoch 0 -> loss: 2.426
epoch 1 -> loss: 1.783
epoch 2 -> loss: 1.958
epoch 3 -> loss: 2.271
epoch 4 -> loss: 1.712
epoch 5 -> loss: 1.859
epoch 6 -> loss: 0.989
epoch 7 -> loss: 0.847
epoch 8 -> loss: 1.135
epoch 9 -> loss: 0.809
epoch 10 -> loss: 0.731
epoch 11 -> loss: 1.077
epoch 12 -> loss: 0.987
epoch 13 -> loss: 0.549
epoch 14 -> loss: 0.008
CPU times: user 12min 48s, sys: 46.9 s, total: 13min 35s
Wall time: 12min 59s


In [24]:
test(net, test_loader)

0.6923720660785013
